In [1]:
import pandas as pd
from pathlib import Path
import os
import warnings
warnings.filterwarnings(action="ignore", category=UserWarning)
import optuna
import pandas as pd
from sklearn import svm
from sklearn import model_selection
import joblib

c:\Users\franc.FJMOYAO\miniconda3\envs\tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from torch.optim import AdamW
from transformers import RobertaForSequenceClassification
from sklearn.metrics import f1_score
import numpy as np
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader

from torch.optim import AdamW
from transformers import RobertaForSequenceClassification, RobertaConfig
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path 
import os 
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch
import utils
from torch.utils.data import Subset, DataLoader

In [3]:
torch.cuda.is_available()

True

In [3]:
#Se leen los datos y se seleccionan las variables que presentan una diferencia
# entre los individuos que presentan estres y los que no
data_path = Path(os.getcwd()).parent / "data" / "silver"

In [4]:
#Loads different feature sets
df = pd.read_csv(os.path.join(data_path,'dreadditCleanTrain.csv'))


In [17]:
df.query("label ==0").text.iloc[100]

"i wanted this addressed and my advocate said the next prosecutor taking over my case was great, so don't worry. now in april of this year i got a call from my attorney from the protection order case. my ex's attorney had contacted her to ask why i hadn't dismissed the case like i'd offered originally. he hadn't brought this up in half a dozen trials, and even provided forged documents to prove my ex's innocence. he had finally requested discovery, seen the actual records and asked for dismissal."

### RoBERTa

In [15]:
# Load the configuration for the model
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
config = RobertaConfig.from_pretrained(model_name, num_labels=2)  # Set to 2 for binary classification
model = RobertaForSequenceClassification.from_pretrained(model_name, config=config, ignore_mismatched_sizes=True)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

config.id2label = {0: "no estres", 1: "estres"}
config.label2id = {"no estres": 0, "estres": 1}

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

In [16]:
# Ejemplo de cómo usar el dataset
dataset = SentimentDataset(df['text'].tolist(), df['label'].tolist(), tokenizer, max_length=128)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [17]:
# Suponiendo que 'model' y 'loader' están definidos y configurados correctamente
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

# Listas para almacenar las métricas
losses = []
f1_scores = []

c:\Users\franc.FJMOYAO\miniconda3\envs\tesis\lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:

# Inicializar el KFold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Convertir labels a numpy para el uso con StratifiedKFold
labels = np.array(df['label'].tolist())

# Ejecutar la validación cruzada
f1_scores = []
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    train_subsampler = Subset(dataset, train_idx)
    val_subsampler = Subset(dataset, val_idx)
    
    train_loader = DataLoader(train_subsampler, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_subsampler, batch_size=16, shuffle=False)

    print(f"Training fold {fold+1}")
    fold_f1 = utils.train_and_evaluate(model, train_loader, val_loader, device)
    f1_scores.append(fold_f1)

print(f"Average F1 across folds: {np.mean(f1_scores):.4f}")

Training fold 1


KeyboardInterrupt: 

In [19]:
import os
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim import AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
import utils

# Definir la ruta de los datos
data_path = Path(os.getcwd()).parent / "data" / "silver"

# Cargar los datos
df = pd.read_csv(data_path / 'dreadditCleanTrain.csv')

# Configurar el modelo
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaForSequenceClassification.from_pretrained(model_name, config=config, ignore_mismatched_sizes=True)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# Etiquetas
config.id2label = {0: "no estres", 1: "estres"}
config.label2id = {"no estres": 0, "estres": 1}

# Definir el Dataset
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Crear el dataset y dataloader
dataset = SentimentDataset(df['text'].tolist(), df['label'].tolist(), tokenizer, max_length=128)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

# Inicializar KFold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
labels = np.array(df['label'].tolist())

# Ejecutar validación cruzada
f1_scores = []
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    train_loader = DataLoader(Subset(dataset, train_idx), batch_size=16, shuffle=True)
    val_loader = DataLoader(Subset(dataset, val_idx), batch_size=16, shuffle=False)
    
    print(f"Training fold {fold+1}")
    fold_f1 = utils.train_and_evaluate(model, train_loader, val_loader, device)
    f1_scores.append(fold_f1)

print(f"Average F1 across folds: {np.mean(f1_scores):.4f}")


c:\Users\franc.FJMOYAO\miniconda3\envs\tesis\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceC

Training fold 1


KeyboardInterrupt: 